<a href="https://colab.research.google.com/github/madelgi/pvd-gan-presentation/blob/master/notebooks/gan_presentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An Introduction to Generative Adversarial Networks

A generative adversarial network (GAN) is a generative machine learning system developed in 2014, with numerous applications in both supervised and unsupervised contexts. Many of the impressive breakthroughs in generative modeling (e.g., [ThisPersonDoesNotExist.com](https://ThisPersonDoesNotExist.com)) use a GAN-based system behind the scenes. By the end of this talk, I hope to cover:

1. What is a generative model?
2. How do GANs work (in broad strokes)?
3. How would you go about building a simple GAN?
4. What are some examples of the current state-of-the-art in GANs?


## Preliminaries 

### Generative vs. Discriminative Models

Broadly, statistical models can be grouped into two categories:

1. **Discriminative**: A discriminative model learns the boundary between classes. Mathematically, we are learning the *conditional* probability distribution $P(Y|X)$.
2. **Generative**: A generative model learns how the data is actually distributed, rather than just the boundaries between different classes. Mathematically, this is equivalent to learning the *joint* probability distribution $P(X, Y)$.

Consider the example [here][1]

[1]: <https://stackoverflow.com/questions/879432/what-is-the-difference-between-a-generative-and-a-discriminative-algorithm>

### Neural networks 

TODO this section can be as long or as short as needed. Ideally it would be short, so we can focus more on GANs themselves

## How Do GANs Work?

GANs consists of both a generative and discriminative component.


## Building a Simple GAN (and Training Pitfalls)

In this section, we lay out the architecture of a very simple GAN, and explore some of the training difficulties that arise with GANs. We are going to be following [this](https://blog.paperspace.com/implementing-gans-in-tensorflow/) tutorial, but our implementation will be in Keras.


### Architecture

As discussed previously, our GAN consists of two components: the generator and discriminator. The generator is responsible for 

### Training: Initial Attempt and Results





### Optimization Techniques

## Current State of the Art

## References

[1]: https://stackoverflow.com/questions/879432/what-is-the-difference-between-a-generative-and-a-discriminative-algorithm